In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)

# reflect the tables


In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement=Base.classes.measurement
station=Base.classes.station


In [ ]:
# Create our session (link) from Python to the DB
session=Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date_string = session.query(func.max(measurement.date)).scalar()
recent_date = dt.datetime.strptime(recent_date_string, '%Y-%m-%d')


In [ ]:
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(measurement.date, measurement.prcp).filter(measurement.date >= prev_year).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date', 'precipitation'])

# Sort the dataframe by date
df = df.sort_values("date")

# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x='date', y='precipitation', rot=90)
plt.xlabel("Date")
plt.ylabel("Inches")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
summary_stats = df['precipitation'].describe()
summary_stats


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count =session.query(measurement.station).distinct().count()
station_count


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_activity = session.query(measurement.station,func.count(measurement.station)).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
for station in station_activity:
    print(f"Station: {station[0]}, Count: {station[1]}")


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_data = session.query(func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)).filter(measurement.station == 'USC00519281')

In [ ]:

print(f'min temp:{most_active_data[0][0]}, max temp:{most_active_data[0][1]}, average temp:{most_active_data[0][2]}')

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
latest_date = session.query(func.max(measurement.date)).filter(measurement.station == 'USC00519281').scalar()
latest_date = dt.datetime.strptime(latest_date, '%Y-%m-%d')
one_year_ago = latest_date - dt.timedelta(days=365)

In [ ]:
tobs_data = session.query(measurement.date, measurement.tobs) \
                   .filter(measurement.station == 'USC00519281') \
                   .filter(measurement.date >= one_year_ago) \
                   .all()

# create a DataFrame from the TOBS data
tobs_df = pd.DataFrame(tobs_data, columns=['date', 'tobs'])

In [ ]:
plt.hist(tobs_df['tobs'], bins=12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title(f'Temperature Observations for Station USC00519281\n{one_year_ago.date()} to {latest_date}')

# Close session

In [ ]:
# Close Session
session.close()